In [1]:
from pathlib import Path
import json, math
import numpy as np

# --- inputs ---
RX_POS_TXT = Path("../NeRAF/data/RAF/EmptyRoom/metadata/all_rx_pos.txt")
TX_POS_TXT = Path("../NeRAF/data/RAF/EmptyRoom/metadata/all_tx_pos.txt")
RUN_DIR_JSON = Path("./outputs/EmptyRoom/images-jpeg-1k/nerfacto/2025-11-04_172911/dataparser_transforms.json")

OUT_JSON = Path("camera_path.json")

# Viewer header
DEFAULT_FOV = 100.0
ASPECT = 1.5
RENDER_H, RENDER_W = 256,256
SECONDS = 999
IS_CYCLE = False
SMOOTHNESS = 0
CAMERA_TYPE = "perspective"

GLOBAL_UP = np.array([0.0, 0.0, 1.0], dtype=float)  # Z-up

# Control how many you keep
LIMIT = None     # None = all
STRIDE = 1       # keep every STRIDE-th pair (e.g., 5)

# ---------- helpers ----------
def float_no_sci(x, ndigits=12):
    s = f"{x:.{ndigits}f}"
    if "." in s:
        s = s.rstrip("0").rstrip(".")
        if s == "-0":
            s = "0"
        if "." not in s:
            s += ".0"
    return float(s)

def matrix_row_major_list(m4):
    return [float_no_sci(m4[i, j]) for i in range(4) for j in range(4)]

def load_dataparser_transform(run_dir_json):
    with open(run_dir_json, "r") as f:
        meta = json.load(f)
    T_3x4 = np.array(meta["transform"], dtype=float)  # (3,4)
    s = float(meta["scale"])
    R = T_3x4[:, :3]
    t = T_3x4[:, 3]
    return R, t, s

def normalize(v, eps=1e-9):
    n = np.linalg.norm(v)
    return v / n if n > eps else v * 0.0

def build_upright_look_at(cam_pos, target_pos, global_up=np.array([0,0,1.0])):
    f = normalize(target_pos - cam_pos)
    r = np.cross(global_up, f)
    if np.linalg.norm(r) < 1e-6:
        r = np.cross(np.array([1.0,0.0,0.0]), f)
        if np.linalg.norm(r) < 1e-6:
            r = np.array([0.0,1.0,0.0])
    r = normalize(r)
    u = np.cross(f, r)
    c2w = np.eye(4, dtype=float)
    c2w[:3, 0] = r
    c2w[:3, 1] = u
    c2w[:3, 2] = f
    c2w[:3, 3] = cam_pos
    return c2w

def parse_rx_all(path):
    """Yield RX positions: each line is y,z,x -> xyz; skip NaNs."""
    with open(path, "r") as f:
        for line in f:
            parts = [p.strip() for p in line.strip().split(",")]
            if len(parts) < 3: 
                continue
            try:
                y = float(parts[0]); z = float(parts[1]); x = float(parts[2])
            except ValueError:
                continue
            if all(math.isfinite(v) for v in (x,y,z)):
                yield np.array([x,y,z], dtype=float)

def parse_tx_all(path):
    """Yield TX quaternion (HypA yzxW -> xyzW) + TX position (yzx -> xyz)."""
    with open(path, "r") as f:
        for line in f:
            parts = [p.strip() for p in line.strip().split(",")]
            if len(parts) < 7:
                continue
            try:
                qy = float(parts[0]); qz = float(parts[1]); qx = float(parts[2]); qw = float(parts[3])  # yzxW
                py = float(parts[4]); pz = float(parts[5]); px = float(parts[6])                      # yzx
            except ValueError:
                continue
            if not all(math.isfinite(v) for v in (qx,qy,qz,qw,px,py,pz)):
                continue
            quat_xyzW = (qx, qy, qz, qw)
            pos_xyz   = np.array([px, py, pz], dtype=float)
            yield quat_xyzW, pos_xyz

def world_to_nerf_point(p_world, R, t, s):
    return s * (R @ p_world + t)

# ---------- main ----------
R_dp, t_dp, s_dp = load_dataparser_transform(RUN_DIR_JSON)

rx_iter = parse_rx_all(RX_POS_TXT)
tx_iter = parse_tx_all(TX_POS_TXT)

camera_entries = []
count_total = 0
count_kept = 0

for idx, (rx_p, tx_tuple) in enumerate(zip(rx_iter, tx_iter)):
    count_total += 1
    if idx % STRIDE != 0:
        continue
    _, tx_p = tx_tuple  # quat parsed but unused for upright mode
    # world -> NeRF
    rx_n = world_to_nerf_point(rx_p, R_dp, t_dp, s_dp)
    tx_n = world_to_nerf_point(tx_p, R_dp, t_dp, s_dp)
    # upright look-at
    c2w = build_upright_look_at(rx_n, tx_n, GLOBAL_UP)
    camera_entries.append({
        "camera_to_world": matrix_row_major_list(c2w),
        "fov": float_no_sci(DEFAULT_FOV),
        "aspect": float_no_sci(ASPECT),
    })
    count_kept += 1
    if LIMIT is not None and count_kept >= LIMIT:
        break

camera_path = {
    "default_fov": float_no_sci(DEFAULT_FOV),
    "default_transition_sec": 2,
    "camera_type": CAMERA_TYPE,
    "render_height": RENDER_H,
    "render_width": RENDER_W,
    "seconds": SECONDS,
    "is_cycle": IS_CYCLE,
    "smoothness_value": SMOOTHNESS,
    "camera_path": camera_entries,
}

with open(OUT_JSON, "w") as f:
    json.dump(camera_path, f, ensure_ascii=False, indent=2)

print(f"Wrote {OUT_JSON.resolve()} with {count_kept} / {count_total} paired cameras (stride={STRIDE}, limit={LIMIT}).")

Wrote /media/scratch/projects/labuser/msc_user/MoNezami/ReverbRAG/camera_path.json with 47484 / 47484 paired cameras (stride=1, limit=None).
